In [ ]:
# example from https://developers.google.com/earth-engine/apidocs/ee-classifier-amnhmaxent#colab-python

"""Demonstrates the ee.Classifier.amnhMaxent method."""

import ee


# Authenticates to the Earth Engine servers.
ee.Authenticate()

In [3]:

# Initializes the client library.
ee.Initialize()

print("setting up training data...")
# Create some sample species presence/absence training data.
# TODO: use real data
training_data = ee.FeatureCollection([
    # Species present points.
    ee.Feature(ee.Geometry.Point([-122.39567, 38.02740]), {'presence': 1}),
    ee.Feature(ee.Geometry.Point([-122.68560, 37.83690]), {'presence': 1}),
    # Species absent points.
    ee.Feature(ee.Geometry.Point([-122.59755, 37.92402]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.47137, 37.99291]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.52905, 37.85642]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.03010, 37.66660]), {'presence': 0})
])

print("getting image(s)...")
# Import a Landsat 8 image and select the reflectance bands.
# TODO: how to use multiple images here? use collection? 
# NOTE: collection(s) used should be (monthly?) means
# TODO: what about multiple collections?
image = (ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20200606')
         .select(['SR_B[1-7]'])
         .multiply(0.0000275).add(-0.2))  # Apply scaling factors.

print("sampling image(s)...")
# Sample the image at the location of the points.
training = image.sampleRegions(**{
    'collection': training_data,
    'scale': 30
})

print("training classifier...")
# Define and train a Maxent classifier from the image-sampled points.
classifier = ee.Classifier.amnhMaxent().train(**{
    'features': training,
    'classProperty': 'presence',
    'inputProperties': image.bandNames()
})

# Classify the image using the Maxent classifier.
image_classified = image.classify(classifier)

print("done!")
display(image_classified)

setting up training data...
getting image(s)...
sampling image(s)...
training classifier...
done!


In [20]:
# Display the thumbnail 
from IPython.display import Image
Image(
    url=image_classified.getThumbURL({
        # 'min': 0, 'max': 2000, 
        'dimensions': 2000, 
        #'region': roi,
        # 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
        'bands':['probability']
    })
)
